In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch 
import torch.nn as nn
import torch.nn.functional as F
import time
import math
# 数据预处理相关函数
from Reader import Footscan_reader
from Reader import gait_data_reader
import random
import pickle
LEARING_RATE=0.001
BATCH_SIZE=100
INPUT_FEATURES=5
HIDDEN_FEATURES_LIST=[32,64]
KERNEL_SIZE_LIST=[(2,2),(3,3),(4,4),(3,3)]
EPOCH_NUM=200
DEVICE=torch.device('cuda')
TIME_STEP=202
footscan_reader = Footscan_reader()
gaitdata_reader=gait_data_reader()


## 1.read the data


In [2]:
info_dic=pickle.load(open('info_dic.pkl','rb'))
output_info=pickle.load(open('../../processed_data/final_output.pkl','rb'))
input_array=np.load('../../processed_data/final_input.npy')
input_array=np.expand_dims(input_array,2)

print('load all data from data buffer file')
print('\n','*'*50,'\n')

load all data from data buffer file

 ************************************************** 



In [4]:
visit_list=gaitdata_reader.get_visit_list(output_info,info_dic)
print(input_array.shape)
group_record=[]
for i in range(len(output_info)):
    group_record.append(output_info[i].group)
print(pd.value_counts(group_record))

该部分数据中共有 20 位患者，平均每位患者有3.45次回访，共计69次,平均每次回访测试了6.565217391304348趟，共计453趟

 

(453, 100, 1, 60, 63)
-24    78
-48    75
-36    74
-12    69
 36    48
 24    42
 48    34
 12    33
dtype: int64


## 2.shuffle the data and turn to tensor format`

In [7]:
valid_input=[]
valid_moment=[]
valid_delta=[]
for i in range(input_array.shape[0]):
    valid_input.append(input_array[i,:,:,:,:])
    valid_moment.append(output_info[i].norm_data)
    valid_delta.append(output_info[i].delta)


#决定训练的task是lacld还是racld
this_input=valid_input
this_moment=valid_moment
this_delta=valid_delta


print(len(this_input))


453


In [8]:
# turn the label to tensor format
label_list=[]
for i in range(len(this_moment)):
    moment=this_moment[i]
    for j in range(100):
        label_list.append(moment[j,3])
print(len(label_list))
LABEL=torch.stack(label_list)
LABEL=torch.unsqueeze(LABEL,1)
#对label放大10倍，这样就避免了doubel和float的精度问题
LABEL=LABEL*100
print(LABEL.shape)
print(torch.mean(LABEL))

45300
torch.Size([45300, 1])
tensor(7.1693)


In [10]:
#处理input——每一帧的通道扩充到5——this frame, last frame, dif_before,next_frame,dif_next
#同时要先把dif这个数据扩大10倍，使其与这一帧的数据同一个数量级，然后再把DATA的数字统一都都扩大100倍，使得和膝关节力矩一个数量级
input_list=[]
for i in range(len(this_input)):
    pressure_video=torch.from_numpy(this_input[i])
    t,c,h,w=pressure_video.shape
    last_pressure=torch.zeros((h,w))
    for j in range(100):
        this_pressure=pressure_video[j,0,:,:]
        if j==99:
            next_pressure=torch.zeros((h,w))
        else:
            next_pressure=pressure_video[j+1,0,:,:]
        #考虑进入时间因素（用delta来表示）
        dif_former=(this_pressure-last_pressure)/ this_delta[i]*10
        dif_latter=(next_pressure-this_pressure)/this_delta[i]*10
        input_list.append(torch.stack([this_pressure,last_pressure,next_pressure,dif_former,dif_latter]))
        last_pressure=this_pressure
DATA=torch.stack(input_list)
DATA=DATA*10
print(DATA.shape)
print(torch.mean(DATA))


torch.Size([45300, 5, 60, 63])
tensor(41.0210, dtype=torch.float64)


In [14]:
#回归模型构建
'''
2023/4/22
处理足压片段来预测步态周期的力矩曲线

模型概况：
    金字塔conv
    flatten
    linear*1

输入输出：
    输入：该时刻的足压图像+该帧的变化量(batch,channel=5,height=180,width=63)
    输出：该时刻的膝关节力矩(batch)
'''
class Net(nn.Module):
    def __init__(self,batch_size,input_features,hidden_features_list,kernel_size_list):
        super(Net,self).__init__()

        self.batch_size = batch_size
        self.input_features = input_features
        self.hidden_features_list=hidden_features_list
        self.kernel_size_list=kernel_size_list

        self.BN1=nn.BatchNorm2d(input_features).to(DEVICE)
        self.pymarid_conv_1=nn.Conv2d(input_features,hidden_features_list[0],kernel_size_list[0],padding='same').to(DEVICE)                              
        self.pymarid_conv_2=nn.Conv2d(input_features,hidden_features_list[0],kernel_size_list[1],padding='same').to(DEVICE)
        self.pymarid_conv_3=nn.Conv2d(input_features,hidden_features_list[0],kernel_size_list[2],padding='same').to(DEVICE)
        self.conv2=nn.Sequential(
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(hidden_features_list[0]*3),
            nn.Conv2d(hidden_features_list[0]*3,hidden_features_list[1],kernel_size_list[3],padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.Linear=nn.Sequential(
            nn.Linear(hidden_features_list[1]*15*15,1600),
            nn.ELU(),
            nn.Linear(1600,512),
            nn.ELU(),
            nn.Linear(512,1),
        ).to(DEVICE)

    def forward(self,x):
        if x.ndim!=4:
            raise ValueError('输入数据ndim不为4')
        else:
            batch,features,height,width= x.shape
            if batch!=self.batch_size or features != self.input_features:
                raise ValueError('输入数据batch size 或者 input features 与model不符')
            else:
                pyramid_conv_output_list=[self.pymarid_conv_1(x),self.pymarid_conv_2(x),self.pymarid_conv_3(x)]
                pyramid_conv_output=torch.cat(pyramid_conv_output_list,dim=1)
                conv2_output=self.conv2(pyramid_conv_output)
                y=self.Linear(conv2_output.reshape(batch,-1))
                return y

model = Net(BATCH_SIZE,INPUT_FEATURES,HIDDEN_FEATURES_LIST,KERNEL_SIZE_LIST).to(DEVICE)     
print(model)
a = torch.ones((BATCH_SIZE,INPUT_FEATURES,60,63)).to(DEVICE)
b = model(a)
print(b.shape)

Net(
  (BN1): BatchNorm2d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pymarid_conv_1): Conv2d(5, 32, kernel_size=(2, 2), stride=(1, 1), padding=same)
  (pymarid_conv_2): Conv2d(5, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (pymarid_conv_3): Conv2d(5, 32, kernel_size=(4, 4), stride=(1, 1), padding=same)
  (conv2): Sequential(
    (0): ReLU()
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(96, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (Linear): Sequential(
    (0): Linear(in_features=14400, out_features=1600, bias=True)
    (1): ELU(alpha=1.0)
    (2): Linear(in_features=1600, out_features=512, bias=True)
    (3): ELU(alpha=1.0)
    (4): Linear(in_features=512, out_features=1, bias=True)
  )
)
tor

In [15]:
print(type(DATA))

<class 'torch.Tensor'>


In [17]:
#find the special data with 10k loss 

print(DATA.shape)
print(LABEL.shape)
CRITERION = nn.MSELoss(reduction='mean')
x_list=torch.split(DATA,BATCH_SIZE,dim=0)
y_list=torch.split(LABEL,BATCH_SIZE,dim=0)
print(len(x_list),len(y_list))

for i in range(len(x_list)):
    pre=model(x_list[i].to(DEVICE).float()).float()
    y=y_list[i].to(DEVICE)
    y=y.float()
    loss=CRITERION(pre,y)
    if loss>10000:
        print(loss)
        print(i)



torch.Size([45300, 5, 60, 63])
torch.Size([45300, 1])
453 453
tensor(8888468., device='cuda:0', grad_fn=<MseLossBackward0>)
397
